In [1]:
import matplotlib.colors as mcolors
import matplotlib.animation as ma
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from itertools import product
import pandas as pd
import numpy as np
import numba as nb
import imageio
import os
import shutil

%matplotlib inline
%config InlineBackend.figure_format = "retina"

new_cmap = mcolors.LinearSegmentedColormap.from_list(
    "new", plt.cm.jet(np.linspace(0, 1, 256)) * 0.85, N=256
)

@nb.njit
def colors_idx(phaseTheta):
    return np.floor(256 - phaseTheta / (2 * np.pi) * 256).astype(np.int32)

import seaborn as sns

sns.set_theme(
    style="ticks", 
    font_scale=1.1, rc={
    'figure.figsize': (6, 5),
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'grid.color': '#dddddd',
    'grid.linewidth': 0.5,
    "lines.linewidth": 1.5,
    'text.color': '#000000',
    'figure.titleweight': "bold",
    'xtick.color': '#000000',
    'ytick.color': '#000000'
})

plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'STIXGeneral'

from main import *
from multiprocessing import Pool
import pandas as pd

colors = ["#403990", "#80A6E2", "#FBDD85", "#F46F43", "#CF3D3E"]
cmap = mcolors.LinearSegmentedColormap.from_list("my_colormap", colors)
cmap_r = mcolors.LinearSegmentedColormap.from_list("my_colormap", colors[::-1])

SAVE_PATH = r"E:\MS_ExperimentData\general"
MP4_PATH = r"E:\MS_ExperimentData\mp4"

In [2]:
class1, class2 = (
    np.concatenate([np.ones(500), np.zeros(500)]).astype(bool), 
    np.concatenate([np.zeros(500), np.ones(500)]).astype(bool)
)
scale = 35
width = 0.003

def draw_mp4(sa: StateAnalysis, fileName: str, mp4Path: str = MP4_PATH, step: int = 1, tailCounts: int = None):
    
    def plot_frame(i):
        pbar.update(1)
        fig.clear()
        ax = plt.subplot(1, 1, 1)
        singlePositionX, singlePhaseTheta, _ = sa.get_state(index=i)

        plt.scatter(
            singlePositionX[class1, 0], singlePositionX[class1, 1],
            color="#F8B08E", # edgecolors="black"
        )
        plt.scatter(
            singlePositionX[class2, 0], singlePositionX[class2, 1],
            color="#9BD5D5", # edgecolors="black"
        )
        ax.quiver(
            singlePositionX[class1, 0], singlePositionX[class1, 1],
            np.cos(singlePhaseTheta[class1]), np.sin(singlePhaseTheta[class1]), color='#F16623', 
            scale=scale, width=width
        )
        ax.quiver(
            singlePositionX[class2, 0], singlePositionX[class2, 1],
            np.cos(singlePhaseTheta[class2]), np.sin(singlePhaseTheta[class2]), color='#49B2B2', 
            scale=scale, width=width
        )

        strengthLambda = np.round(sa.model.strengthLambda, 3)
        distanceD0 = np.round(sa.model.distanceD0, 2)
        ax.set_title(f"$\lambda={strengthLambda},\ d_0={distanceD0}$", fontweight="bold")
        ax.set_xlim(0, 10)
        ax.set_ylim(0, 10)
        ax.set_xticks([])
        ax.set_yticks([])
        plt.tight_layout()

    fig, _ = plt.subplots(1, 1, figsize=(4, 4))
    TNum = sa.TNum
    startIdx = TNum - tailCounts * step if tailCounts else 0
    frames = np.arange(startIdx, TNum, step)
    pbar = tqdm(total=len(frames))
    ani = ma.FuncAnimation(fig, plot_frame, frames=frames, interval=40, repeat=False)
    ani.save(f"{mp4Path}/{fileName}.mp4", dpi=200)
    plt.close()
    pbar.close()

In [3]:
omegaMins = np.linspace(0.1, 0.5, 30)
phaseLags = np.linspace(-1, 1, 30) * np.pi
randomSeed = 9
strengthLambda = 0.15
distanceD0 = 0.5
deltaOmega = 1
halfAgentsNum = 500
Homega = np.block([
    [np.ones((halfAgentsNum, halfAgentsNum)), np.zeros((halfAgentsNum, halfAgentsNum))],
    [np.zeros((halfAgentsNum, halfAgentsNum)), np.ones((halfAgentsNum, halfAgentsNum))]
])

SAVE_PATH = r"E:\MS_ExperimentData\general"

In [4]:
phaseLag = phaseLags[7]
omegaMin = omegaMins[0]

model = ChiralInducedPhaseLag(strengthLambda=strengthLambda, distanceD0=distanceD0, boundaryLength=10,
                            phaseLag=phaseLag, agentsNum=1000,
                            omegaMin=omegaMin, deltaOmega=deltaOmega,
                            savePath=SAVE_PATH, dt=0.02,
                            tqdm=True, overWrite=True, randomSeed=randomSeed)

sa = StateAnalysis(model)

In [6]:
draw_mp4(sa, str(model), tailCounts=None)

  0%|          | 0/6002 [00:00<?, ?it/s]